# Proyecto 2
### Data science
- Abner Gardia 21285
- Esteban Donis 21610
- Daniel Gomez 21429

In [138]:
# project 2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Inciso 2
Analice el problema planteado y los datos.

In [139]:
train = pd.read_csv('./data/train.csv')

In [140]:
train.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


In [141]:
train.describe()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
count,36765,36765,36765,36765,36765
unique,36765,4191,36691,7,3
top,0013cc385424,91B1F82B2CF1,Summer projects should be student-designed,Evidence,Adequate
freq,1,23,14,12105,20977


In [142]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36765 entries, 0 to 36764
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   discourse_id             36765 non-null  object
 1   essay_id                 36765 non-null  object
 2   discourse_text           36765 non-null  object
 3   discourse_type           36765 non-null  object
 4   discourse_effectiveness  36765 non-null  object
dtypes: object(5)
memory usage: 1.4+ MB


### Inciso 3
Describa las tareas de limpieza y preprocesamiento que llevó a cabo.

### Cambios columna discourse_text

In [143]:
train['discourse_text'] = train['discourse_text'].str.upper()

train['discourse_text'] = train['discourse_text'].str.replace(r"N'T", ' NOT', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"'S", ' IS', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"ITS", 'IT IS', regex=True)
train['discourse_text'] = train['discourse_text'].str.replace(r"'RE", ' ARE', regex=True)

In [144]:
train['discourse_text'] = train['discourse_text'].str.replace(r"[^A-Z0-9 ]", ' ', regex=True)

In [145]:
import nltk
from nltk.corpus import stopwords
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/estebandonis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [146]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenize the text
    words = text.split()
    # Remove stopwords and apply stemming
    processed_words = [word for word in words]
    # Reassemble the text
    return ' '.join(processed_words)

In [147]:
train['discourse_text'] = train['discourse_text'].apply(preprocess_text)

### Cambios columna discourse_type

In [148]:
train['discourse_type'] = train['discourse_type'].str.upper()
train['discourse_type'] = train['discourse_type'].str.replace(r"[^A-Z0-9 ]", ' ', regex=True)

### Cambios columna discourse_effectiveness

In [149]:
train['discourse_effectiveness'] = train['discourse_effectiveness'].str.upper()
train['discourse_effectiveness'] = train['discourse_effectiveness'].str.replace(r"[^A-Z0-9 ]", ' ', regex=True)